In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

2022-11-29 15:57:44.849997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 15:57:45.761438: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shimizu-reo/cudas/lib64:/usr/local/cuda/lib64:/usr/local/cuda/lib64:
2022-11-29 15:57:45.761558: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/shimizu-reo/cudas/lib64:/usr/local/cuda/l

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp

import neural_tangents as nt
from neural_tangents import stax

import tensorflow_datasets as tfds
import scipy as sp
from tqdm import tqdm

In [3]:
num_classes = 10

In [4]:
train_ds, test_ds = tfds.load('mnist:3.*.*', split=['train', 'test'])

def preprocess(x):
    image, label = x['image'], x['label']
    image = tf.cast(image, tf.float64)
    image = image / 255.
    image = tf.reshape(image, (-1,))
    return image, label

train_ds = train_ds.map(preprocess).as_numpy_iterator()
test_ds = test_ds.map(preprocess).as_numpy_iterator()

def load_x_y(ds):
    x_list, y_list = [], []
    for x, y in ds:
        x = x / np.linalg.norm(x)
        y = np.eye(num_classes)[y]
        x_list.append(x)
        y_list.append(y)
    return np.stack(x_list, axis=0), np.stack(y_list, axis=0)

train_x, train_y = load_x_y(train_ds)
test_x, test_y = load_x_y(test_ds)

In [9]:
def exp(train_x, test_x, train_y, test_y, mid_channels):
    np.random.seed(42)
    train_length = 20000
    test_length = 10000
    inds = np.random.randint(0, train_x.shape[0], train_length)
    train_x = train_x[inds]
    train_y = train_y[inds]
    inds = np.random.randint(0, test_x.shape[0], test_length)
    test_x = test_x[inds]
    test_y = test_y[inds]

    # _,_,kernel_fn = stax.serial(stax.Dense(mid_channels), stax.Relu(), stax.Dense(num_classes))

    init_fn, apply_fn, kernel_fn = stax.serial(
        stax.Dense(mid_channels, parameterization='ntk', b_std=None),
        stax.Relu(),
        stax.Dense(num_classes, parameterization='ntk', b_std=None)
    )
    kernel_fn = jax.jit(kernel_fn, static_argnums=(2,))

    all_x = np.concatenate([train_x, test_x], axis=0)
    K = kernel_fn(all_x, all_x, 'ntk')

    K_train = K[:train_length, :train_length]
    K_test = K[:train_length, train_length:]

    def predict():
        return K_test.T @ jnp.linalg.inv(K_train + K_train.shape[0] * 1e-8 * jnp.eye(K_train.shape[0])) @ train_y

    def evaluate():
        pred = predict()
        y_pred = jnp.argmax(pred, axis=-1)
        gt = jnp.argmax(test_y, axis=-1)
        accuracy = jnp.mean(y_pred == gt)
        return y_pred, accuracy

    _, acc = evaluate()

    print(mid_channels, acc)

In [10]:
for mid_channels in [256, 512, 1024, 2048, 4096]:
    exp(train_x, test_x, train_y, test_y, mid_channels)

256 0.9723
512 0.9723
1024 0.9723
2048 0.9723
4096 0.9723
